In [278]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [279]:
# !pip install pyarrow
# !pip install fastparquet
# !pip install xgboost


In [280]:
# Hei

In [281]:
def list_directory_tree_with_os_walk(starting_directory):
    for root, directories, files in os.walk(starting_directory):
        print(f"Directory: {root}")
        for file in files:
            print(f"  File: {file}")

list_directory_tree_with_os_walk('.')

Directory: .
  File: .DS_Store
  File: data_csv.zip
  File: my_first_submission.csv
  File: prediction.ipynb
  File: preprocessing.ipynb
  File: Readme.md
  File: sample_submission.csv
  File: test.csv
Directory: .\A
  File: train_targets.parquet
  File: X_test_estimated.parquet
  File: X_train_estimated.parquet
  File: X_train_observed.parquet
Directory: .\B
  File: train_targets.parquet
  File: X_test_estimated.parquet
  File: X_train_estimated.parquet
  File: X_train_observed.parquet
Directory: .\C
  File: train_targets.parquet
  File: X_test_estimated.parquet
  File: X_train_estimated.parquet
  File: X_train_observed.parquet
Directory: .\cleaned_data
  File: x_test_combined.csv
  File: x_train_combined.csv
  File: y_train_combined.csv
Directory: .\data_csv
Directory: .\data_csv\A
  File: train_targets.csv
  File: X_test_estimated.csv
  File: X_train_estimated.csv
  File: X_train_observed.csv
Directory: .\data_csv\B
  File: train_targets.csv
  File: X_test_estimated.csv
  File: X_tr

In [282]:
train_a = pd.read_parquet('A/train_targets.parquet')
train_b = pd.read_parquet('B/train_targets.parquet')
train_c = pd.read_parquet('C/train_targets.parquet')

In [283]:
X_train_estimated_a = pd.read_parquet('A/X_train_estimated.parquet')
X_train_estimated_b = pd.read_parquet('B/X_train_estimated.parquet')
X_train_estimated_c = pd.read_parquet('C/X_train_estimated.parquet')

In [284]:
X_train_observed_a = pd.read_parquet('A/X_train_observed.parquet')
X_train_observed_b = pd.read_parquet('B/X_train_observed.parquet')
X_train_observed_c = pd.read_parquet('C/X_train_observed.parquet')


In [285]:
X_test_estimated_a = pd.read_parquet('A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('B/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('C/X_test_estimated.parquet')


In [286]:
# # Plotting a single feature
# fig, axs = plt.subplots(3, 1, figsize=(20, 10), sharex=True)
# feature_name = 'cloud_base_agl:m'
# X_train_observed_a[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[0], title='Train/Test A', color='red')
# X_train_estimated_a[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[0], title='Train/Test A', color='blue')
# X_test_estimated_a[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[0], title='Train/Test  A', color='green')

# X_train_observed_b[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[1], title='Train/Test  B', color='red')
# X_train_estimated_b[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[1], title='Train/Test  B', color='blue')
# X_test_estimated_b[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[1], title='Train/Test  B', color='green')

# X_train_observed_c[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[2], title='Train/Test  C', color='red')
# X_train_estimated_c[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[2], title='Train/Test  C', color='blue')
# X_test_estimated_c[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[2], title='Train/Test  C', color='green')

# train_a[['time', 'pv_measurement']].set_index('time').plot(ax=axs[0], title='y_values', color='yellow')

In [287]:
X_train_merged_a = pd.concat([X_train_observed_a,X_train_estimated_a])
X_train_merged_b = pd.concat([X_train_observed_b,X_train_estimated_b])
X_train_merged_c = pd.concat([X_train_observed_c,X_train_estimated_c])

In [288]:
# fig, axs = plt.subplots(3, 1, figsize=(20, 10), sharex=True)
# feature_name = 'cloud_base_agl:m'
# X_train_merged_a[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[0], title='Train/Test A', color='red')
# X_test_estimated_a[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[0], title='Train/Test  A', color='green')

# X_train_merged_b[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[1], title='Train/Test  B', color='red')
# X_test_estimated_b[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[1], title='Train/Test  B', color='green')

# X_train_merged_c[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[2], title='Train/Test  C', color='red')
# X_test_estimated_c[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[2], title='Train/Test  C', color='green')

# train_a[['time', 'pv_measurement']].set_index('time').plot(ax=axs[0], title='y_values', color='yellow')

In [289]:
# Calculating from 15-minute intervals to hourly intervals based on different aggregation methods

X_train_merged_a.set_index('date_forecast', inplace=True)
X_train_merged_b.set_index('date_forecast', inplace=True)
X_train_merged_c.set_index('date_forecast', inplace=True)

X_test_estimated_a.set_index('date_forecast', inplace=True)
X_test_estimated_b.set_index('date_forecast', inplace=True)
X_test_estimated_c.set_index('date_forecast', inplace=True)

aggregation_methods = {
    'date_calc' : 'max',
    'absolute_humidity_2m:gm3': 'mean',
    'air_density_2m:kgm3': 'mean',
    'ceiling_height_agl:m': 'mean',
    'clear_sky_energy_1h:J': 'sum',
    'clear_sky_rad:W': 'mean',
    'cloud_base_agl:m': 'mean',
    'dew_or_rime:idx': 'max',
    'dew_point_2m:K': 'mean',
    'diffuse_rad:W': 'mean',
    'diffuse_rad_1h:J': 'sum',
    'direct_rad:W': 'mean',
    'direct_rad_1h:J': 'sum',
    'effective_cloud_cover:p': 'mean',
    'elevation:m': 'mean',
    'fresh_snow_12h:cm': 'sum',
    'fresh_snow_1h:cm': 'sum',
    'fresh_snow_24h:cm': 'sum',
    'fresh_snow_3h:cm': 'sum',
    'fresh_snow_6h:cm': 'sum',
    'is_day:idx': 'max',
    'is_in_shadow:idx': 'max',
    'msl_pressure:hPa': 'mean',
    'precip_5min:mm': 'sum',
    'precip_type_5min:idx': 'max',
    'pressure_100m:hPa': 'mean',
    'pressure_50m:hPa': 'mean',
    'prob_rime:p': 'mean',
    'rain_water:kgm2': 'sum',
    'relative_humidity_1000hPa:p': 'mean',
    'sfc_pressure:hPa': 'mean',
    'snow_density:kgm3': 'mean',
    'snow_depth:cm': 'mean',
    'snow_drift:idx': 'max',
    'snow_melt_10min:mm': 'sum',
    'snow_water:kgm2': 'sum',
    'sun_azimuth:d': 'mean',
    'sun_elevation:d': 'mean',
    'super_cooled_liquid_water:kgm2': 'mean',
    't_1000hPa:K': 'mean',
    'total_cloud_cover:p': 'mean',
    'visibility:m': 'mean',
    'wind_speed_10m:ms': 'mean',
    'wind_speed_u_10m:ms': 'mean',
    'wind_speed_v_10m:ms': 'mean',
    'wind_speed_w_1000hPa:ms': 'mean'
}

X_train_a_estimated_hourly = X_train_merged_a.resample('H').agg(aggregation_methods)
X_train_b_estimated_hourly = X_train_merged_b.resample('H').agg(aggregation_methods)
X_train_c_estimated_hourly = X_train_merged_c.resample('H').agg(aggregation_methods)

X_test_a_estimated_hourly = X_test_estimated_a.resample('H').agg(aggregation_methods)
X_test_b_estimated_hourly = X_test_estimated_b.resample('H').agg(aggregation_methods)
X_test_c_estimated_hourly = X_test_estimated_c.resample('H').agg(aggregation_methods)

X_train_a_estimated_hourly.reset_index(inplace=True)
X_train_b_estimated_hourly.reset_index(inplace=True)
X_train_c_estimated_hourly.reset_index(inplace=True)

X_test_a_estimated_hourly.reset_index(inplace = True)
X_test_b_estimated_hourly.reset_index(inplace = True)
X_test_c_estimated_hourly.reset_index(inplace = True)

In [290]:
# Identify the indices of the rows with NaN values in the 'pv_measurement' column
nan_indices_a = train_a[train_a['pv_measurement'].isna()].index
nan_indices_b = train_b[train_b['pv_measurement'].isna()].index
nan_indices_c = train_c[train_c['pv_measurement'].isna()].index

# Drop these indices from y_train
train_a = train_a.drop(nan_indices_a)
train_b = train_b.drop(nan_indices_b)
train_c = train_c.drop(nan_indices_c)


In [291]:
def synchronize_x_train_with_targets(x_train, y_train):
    # Convert date columns to datetime format for easier comparison
    x_train['date_forecast'] = pd.to_datetime(x_train['date_forecast'])
    y_train['time'] = pd.to_datetime(y_train['time'])
    
    # Filter x_train rows where 'date_forecast' matches with 'time' in y_train
    x_train_synced = x_train[x_train['date_forecast'].isin(y_train['time'])]
    
    return x_train_synced

def filter_y_train_to_match_x(x_train_synced, y_train):
    # Filter y_train rows where 'time' matches with 'date_forecast' in x_train_synced
    y_train_filtered = y_train[y_train['time'].isin(x_train_synced['date_forecast'])]
    
    return y_train_filtered


X_train_a_estimated_hourly = synchronize_x_train_with_targets(X_train_a_estimated_hourly,train_a)
X_train_b_estimated_hourly = synchronize_x_train_with_targets(X_train_b_estimated_hourly,train_b)
X_train_c_estimated_hourly = synchronize_x_train_with_targets(X_train_c_estimated_hourly,train_c)

train_a = filter_y_train_to_match_x(X_train_a_estimated_hourly, train_a)
train_b = filter_y_train_to_match_x(X_train_b_estimated_hourly,train_b)
train_c = filter_y_train_to_match_x(X_train_c_estimated_hourly, train_c)

In [292]:
# Add a 'location' column to each dataframe
X_train_a_estimated_hourly['location'] = 'A'
X_train_b_estimated_hourly['location'] = 'B'
X_train_c_estimated_hourly['location'] = 'C'

X_test_a_estimated_hourly['location'] = 'A'
X_test_b_estimated_hourly['location'] = 'B'
X_test_c_estimated_hourly['location'] = 'C'

# Concatenate all three dataframes
X_train_combined = pd.concat([X_train_a_estimated_hourly, X_train_b_estimated_hourly, X_train_c_estimated_hourly], ignore_index=True)
X_test_combined = pd.concat([X_test_a_estimated_hourly, X_test_b_estimated_hourly, X_test_c_estimated_hourly], ignore_index=True)

X_test_combined.head()


,date_forecast,date_calc,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,location
0,2023-05-01 00:00:00,2023-04-30 07:00:04,4.325,1.28675,912.700012,0.000000,0.000,1061.550049,0.0,271.650024,0.000000,0.00000,0.000000,0.000000,74.949997,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.674988,0.0,0.0,1000.549988,1006.799988,0.0,0.0,80.275002,1013.099976,NaN,0.0,0.0,0.0,0.0,16.026501,-10.5410,0.0,273.799988,74.949997,29907.500000,3.950,2.100,3.35,0.0,A
1,2023-05-01 01:00:00,2023-04-30 07:00:04,4.275,1.28600,1482.099976,0.000000,0.000,1075.100098,0.0,271.450012,0.000000,0.00000,0.000000,0.000000,77.474998,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.150024,0.0,0.0,1000.049988,1006.299988,0.0,0.0,79.824997,1012.599976,NaN,0.0,0.0,0.0,0.0,30.497250,-7.8945,0.0,273.799988,77.474998,29519.074219,3.825,1.925,3.30,0.0,A
2,2023-05-01 02:00:00,2023-04-30 07:00:04,4.150,1.28375,1791.300049,0.000000,0.000,1200.400024,0.0,271.049988,0.000000,0.00000,0.000000,0.000000,88.099998,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1012.674988,0.0,0.0,999.500000,1005.799988,0.0,0.0,78.000000,1012.049988,NaN,0.0,0.0,0.0,0.0,44.517250,-3.8155,0.0,273.849976,88.099998,31009.125000,3.650,1.750,3.20,0.0,A
3,2023-05-01 03:00:00,2023-04-30 07:00:04,4.025,1.28200,2312.875000,40497.699219,11.675,1179.849976,0.0,270.649994,9.375000,67380.90625,2.100000,15061.400391,68.599998,6.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1012.174988,0.0,0.0,998.974976,1005.224976,0.0,0.0,75.625000,1011.525024,NaN,0.0,0.0,0.0,0.0,58.083000,1.4125,0.0,273.899994,68.599998,34552.500000,3.500,1.450,3.15,0.0,A
4,2023-05-01 04:00:00,2023-04-30 07:00:04,3.900,1.28100,2198.299805,566994.375000,76.875,920.049988,0.0,270.375000,47.400002,408838.81250,25.450001,198284.796875,66.300003,6.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1011.724976,0.0,0.0,998.549988,1004.750000,0.0,0.0,74.224998,1011.049988,NaN,0.0,0.0,0.0,0.0,71.341003,7.4685,0.0,273.924988,66.300003,35483.875000,3.325,1.300,3.05,0.0,A


In [293]:
X_train_combined.head()

,date_forecast,date_calc,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,location
0,2019-06-02 22:00:00,NaT,7.700,1.22825,1728.949951,0.000000,0.00,1728.949951,0.0,280.299988,0.000,0.000000,0.00,0.000000,99.074997,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1006.299988,0.0,0.0,993.750000,999.775024,0.0,0.0,71.674995,1005.799988,NaN,0.0,0.0,0.0,0.7,348.036743,-3.77425,0.0,286.225006,100.000000,40386.476562,3.600,-3.575,-0.500,0.0,A
1,2019-06-02 23:00:00,NaT,7.700,1.22350,1689.824951,0.000000,0.00,1689.824951,0.0,280.299988,0.000,0.000000,0.00,0.000000,99.750000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1005.200012,0.0,0.0,992.674988,998.650024,0.0,0.1,68.000000,1004.650024,NaN,0.0,0.0,0.0,0.8,91.980751,-4.35725,0.0,286.899994,100.000000,33770.648438,3.350,-3.350,0.275,0.0,A
2,2019-06-03 00:00:00,NaT,7.875,1.21975,1563.224976,0.000000,0.00,1563.224976,0.0,280.649994,0.000,0.000000,0.00,0.000000,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1004.525024,0.0,0.0,992.000000,997.974976,0.0,0.4,67.949997,1003.950012,NaN,0.0,0.0,0.0,1.6,14.934750,-3.30950,0.0,286.950012,100.000000,13595.500000,3.050,-2.950,0.750,0.0,A
3,2019-06-03 01:00:00,NaT,8.425,1.21800,1283.425049,834.599976,0.75,1283.425049,0.0,281.674988,0.300,2107.100098,0.00,0.000000,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1004.025024,0.0,0.0,991.500000,997.449951,0.0,0.5,73.875000,1003.449951,NaN,0.0,0.0,0.0,2.2,28.630251,-0.82250,0.0,286.750000,100.000000,2321.850098,2.725,-2.600,0.875,0.0,A
4,2019-06-03 02:00:00,NaT,8.950,1.21800,1003.500000,129872.601562,23.10,1003.500000,0.0,282.500000,11.975,88275.796875,0.15,1131.900024,84.875000,6.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1003.099976,0.0,0.0,990.550049,996.500000,0.0,0.4,79.925003,1002.500000,NaN,0.0,0.0,0.0,1.0,41.997501,3.05125,0.0,286.450012,99.224998,11634.799805,2.550,-2.350,0.925,0.0,A


In [294]:
# add location column for each target-dataset
train_a['location'] = 'A'
train_b['location'] = 'B'
train_c['location'] = 'C'

# Concatinate all three datasets
y_train_combined = pd.concat([train_a, train_b, train_c], ignore_index=True)


In [295]:


# Convert to datetime
X_train_combined['date_forecast'] = pd.to_datetime(X_train_combined['date_forecast'])
X_train_combined['date_calc'] = pd.to_datetime(X_train_combined['date_calc'])

# Extract features from date_forecast
X_train_combined['year_forecast'] = X_train_combined['date_forecast'].dt.year
X_train_combined['month_forecast'] = X_train_combined['date_forecast'].dt.month
X_train_combined['day_forecast'] = X_train_combined['date_forecast'].dt.day
X_train_combined['hour_forecast'] = X_train_combined['date_forecast'].dt.hour


# Extract features from date_calc (only if there are non-NaN values)
if X_train_combined['date_calc'].notna().any():
    X_train_combined['year_calc'] = X_train_combined['date_calc'].dt.year
    X_train_combined['month_calc'] = X_train_combined['date_calc'].dt.month
    X_train_combined['day_calc'] = X_train_combined['date_calc'].dt.day
    X_train_combined['hour_calc'] = X_train_combined['date_calc'].dt.hour

# Drop the original date columns
X_train_combined.drop(columns=['date_forecast', 'date_calc'], inplace=True)


# Repeat for the other datasets

# Convert to datetime
y_train_combined['time'] = pd.to_datetime(y_train_combined['time'])

# Extract features from date_forecast
y_train_combined['year'] = y_train_combined['time'].dt.year
y_train_combined['month'] = y_train_combined['time'].dt.month
y_train_combined['day'] = y_train_combined['time'].dt.day
y_train_combined['hour'] = y_train_combined['time'].dt.hour



# Drop the original date columns
y_train_combined.drop(columns=['time'], inplace=True)


# Convert to datetime
X_test_combined['date_forecast'] = pd.to_datetime(X_test_combined['date_forecast'])
X_test_combined['date_calc'] = pd.to_datetime(X_test_combined['date_calc'])

# Extract features from date_forecast
X_test_combined['year_forecast'] = X_test_combined['date_forecast'].dt.year
X_test_combined['month_forecast'] = X_test_combined['date_forecast'].dt.month
X_test_combined['day_forecast'] = X_test_combined['date_forecast'].dt.day
X_test_combined['hour_forecast'] = X_test_combined['date_forecast'].dt.hour


# Extract features from date_calc (only if there are non-NaN values)
if X_test_combined['date_calc'].notna().any():
    X_test_combined['year_calc'] = X_test_combined['date_calc'].dt.year
    X_test_combined['month_calc'] = X_test_combined['date_calc'].dt.month
    X_test_combined['day_calc'] = X_test_combined['date_calc'].dt.day
    X_test_combined['hour_calc'] = X_test_combined['date_calc'].dt.hour

# Drop the original date columns
X_test_combined.drop(columns=['date_forecast', 'date_calc'], inplace=True)




In [296]:
X_test_combined_encoded = pd.get_dummies(X_test_combined['location'], prefix='location')
X_train_combined_encoded = pd.get_dummies(X_train_combined['location'], prefix='location')
y_train_combined_encoded = pd.get_dummies(y_train_combined['location'], prefix='location')

# Drop the original location column
X_test_combined = X_test_combined.drop('location', axis=1)
X_train_combined = X_train_combined.drop('location', axis=1)
y_train_combined = y_train_combined.drop('location', axis=1)

# Concatenate the original dataframe with the encoded dataframe
X_test_combined = pd.concat([X_test_combined, X_test_combined_encoded], axis=1)
X_train_combined = pd.concat([X_train_combined, X_train_combined_encoded], axis=1)
y_train_combined = pd.concat([y_train_combined, y_train_combined_encoded], axis=1)


In [297]:
# Sorting x_train
x_train = X_train_combined.sort_values(by=['location_A', 'location_B', 'location_C', 'year_forecast', 'month_forecast', 'day_forecast', 'hour_forecast'])

# Sorting y_train
y_train = y_train_combined.sort_values(by=['location_A', 'location_B', 'location_C', 'year', 'month', 'day', 'hour'])

# Sorting x_test
x_test = X_test_combined.sort_values(by=['location_A', 'location_B', 'location_C', 'year_forecast', 'month_forecast', 'day_forecast', 'hour_forecast'])


In [298]:
# Drop unwanted columns
columns_to_drop = ['year_forecast', 'month_forecast', 'day_forecast', 'hour_forecast', 
                   'year_calc', 'month_calc', 'day_calc', 'hour_calc']
x_train = x_train.drop(columns=columns_to_drop)
x_test = x_test.drop(columns=columns_to_drop)



In [299]:
output_dir = 'cleaned_data'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the dataframes into the folder
X_train_combined.to_csv(os.path.join(output_dir, 'x_train_combined.csv'), index=False)
X_test_combined.to_csv(os.path.join(output_dir, 'x_test_combined.csv'), index=False)
y_train_combined.to_csv(os.path.join(output_dir, 'y_train_combined.csv'), index=False)

In [300]:
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission

,id,prediction
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
2155,2155,0
2156,2156,0
2157,2157,0
2158,2158,0


In [301]:
# Example, let the predictions be random values
test['prediction'] = np.random.rand(len(test))
sample_submission = sample_submission[['id']].merge(test[['id', 'prediction']], on='id', how='left')
sample_submission.to_csv('my_first_submission.csv', index=False)

NameError: name 'test' is not defined